This notebook runs the analysis of the 'idr0021 Segmentation' notebook for all users and generates the 'Summary from R' table and the ROIs. The first code block just performs some setup tasks, the second will run the analysis.

# Setup

In [ ]:
# Load the libraries
library(romero.gateway)
library(EBImage)

#' Create ROIs from a features table
#'
#' @param features The shape and moments generated by EBImage computeFeatures
#' @return A dataframe specifying the x, y, rx, ry, w, h, theta and text parameters of the ROIs
createROIs <- function(features) {
  rois <- data.frame(x = c(0), y = c(0), rx = c(0), ry = c(0), w = c(0), h = c(0), theta = c(0), text = c('remove'), stringsAsFactors = FALSE)
  for (index in 1:length(features[,1])) {
    x <- features[index,8]
    y <- features[index,9]
    r1 <- features[index,10]
    ecc <- features[index,11]
    r2 <- sqrt(r1^2 * (1 - ecc^2))
    theta <- features[index,12]
    rois <- rbind(rois, c(x, y, r1, r2, NA, NA, -theta, as.character(index)))
  }
  rois <- rois[-1,]
  rownames(rois) <- c()
  rois
}

#' Performs an image segmentation to find the largest ROI of the image
#' and returns some features of interest (area, perimeter and diameter).
#'
#' @param image The image to segement
#' @param channelName The channel to be taken into account
#' @param df The dataframe to add the features to (channelName, imageName, ImageID, ROIINdex, area, perimeter, diameter)
#' @param saveROIs Flag if ROIs should be created and attached to the image (default: FALSE)
#' @return The dataframe with the features
analyzeImage <- function(image, channelName, df, saveROIs = FALSE) {
  # Find the channel index
  chnames <- getChannelNames(image)
  chindex <- match(channelName, chnames, nomatch = 0)
  if (chindex == 0) {
    message (paste("Could not resolve channel name, skipping image ", image@dataobject$getId()))
    return(df)
  }
  
  # Load the pixels
  pixels <- getPixelValues(image, 1, 1, chindex)
  ebi <- EBImage::Image(data = pixels, colormode = 'Grayscale')
  ebi <- normalize(ebi)
  
  # this is our segmentation workflow from above
  threshImg <- thresh(ebi, w=15, h=15, offset=0.1)
  threshImg <- medianFilter(threshImg, size=3)
  threshImg <- fillHull(threshImg)          
  threshImg <- bwlabel(threshImg)
  
  # Calculate the features
  shapes = suppressMessages(computeFeatures.shape(threshImg))
  moments = suppressMessages(computeFeatures.moment(threshImg))
  features <- merge(shapes, moments, by=0, all=TRUE)
  
  if (length(features[,1])>1) {
    # Add the ROIs to the image
    if (saveROIs) {
      rois <- createROIs(features)
      addROIs(image, coords = rois)
    }
    
    # Add the interesting properties (area, perimeter and diameter)
    # of the largest object together with channel name, image name, image id 
    # and roi index to the dataframe
    features <- features[order(-features[,2]),]
    diameter <- features[1,4]*2
    df <- rbind(df, c(channelName, image@dataobject$getName(), image@dataobject$getId(), features[1,1], features[1,2], features[1,3], diameter))
  }
  df
}

analyzeProject <- function(server, projectId, save = FALSE) {
  project <- loadObject(server, "ProjectData", projectId)
  datasets <- getDatasets(project)
  
  result <- data.frame(Channel = c('remove'), ImageName = c('remove'), Image = c(0), ROIIndex = c(0), Area = c(0), Perimeter = c(0), Diameter = c(0), stringsAsFactors = FALSE)
  
  for (dataset in datasets) {
    dsname <- dataset@dataobject$getName()
    if (startsWith(dsname, "CEP120")) # CEP120 data is represented by two datasets (both start with CEP120)
      dsname <- "CEP120"
    images <- getImages(dataset)
    for (image in images) {
      result <- tryCatch({
        analyzeImage(image, dsname, result, saveROIs = save)
      }, warning = function(war) {
        message(paste("WARNING:  ", image@dataobject$getId(),war))
        return(result)
      }, error = function(err) {
        message(paste("ERROR:  ", image@dataobject$getId() ,err))
        return(result)
      }, finally = {
      })
    }
  }
  
  # remove the first row and row names
  result <- result[-1,]
  rownames(result) <- c()
  
  # set the correct datatypes
  result$Channel <- as.factor(result$Channel)
  result$Area <- as.numeric(result$Area)
  result$Perimeter <- as.numeric(result$Perimeter)
  result$Diameter <- as.numeric(result$Diameter)
  result$Image <- as.integer(result$Image)
  
  # set the correct pixel size
  pxSizeInNM <- 40
  result$Area <- result$Area * pxSizeInNM * pxSizeInNM
  result$Perimeter <- result$Perimeter * pxSizeInNM
  result$Diameter <- result$Diameter * pxSizeInNM
  
  # Rename the Channel column to Dataset (for parade)
  colnames(result)[1] <- "Dataset"
  result$Image <- as.integer(result$Image)
  
  if (save) {
    # attach the dataframe to the project
    invisible(attachDataframe(project, result, "Summary from R"))
    
    # Change the namespace (for parade) (workaround as this is not yet
    # implemented in the R gateway itself)
    dataframes <- availableDataframes(project)
    faid <- dataframes$AnnotationID[1]
    fa <- loadObject(server, "omero.gateway.model.FileAnnotationData", as.integer(faid))
    fa@dataobject$setNameSpace("openmicroscopy.org/omero/bulk_annotations")
    gateway <- getGateway(server)
    ctx <- getContext(server)
    dm <- gateway$getFacility(DataManagerFacility$class)
    fa <- dm$saveAndReturnObject(ctx, fa@dataobject)
  }
  result
}

# Run the analysis for all users

In [ ]:
#########################################
# Adjust these parameters accordingly
# before running the cell!
hostname <- 'outreach.openmicroscopy.org'
passwd <- ""
projectName <- "idr0021"
userFrom <- 1
userTo <- 2
# If set to TRUE the analysis will run but no attachements or ROIs will be created
dryrun <- TRUE
#########################################

for (index in userFrom:userTo) {
  user <- paste('user', index, sep='-')
  curTime <- Sys.time()
  message(paste(curTime, user))
  server <- OMEROServer(host = hostname, username=user, password=passwd, port= as.integer(4064))
  server <- connect(server)
  userid <- server@user$getId()
  projs <- getProjects(server)
  for (proj in projs) {
    projName <- proj@dataobject$getName()
    owner <- proj@dataobject$getOwner()$getId()
    if (projName == projectName && owner == userid) {
      message(paste("analyzing project", getOMEROID(proj)))
      df <- analyzeProject(server, getOMEROID(proj), save = !dryrun)
      message(paste(nrow(df), "results gained."))
      break
    }
  }
  disconnect(server)
}
message("Finished.")